In [22]:
import os
import time
import gnupg
import keyring
from binance.spot import Spot
import numpy as np
import pandas as pd

In [23]:
gpg = gnupg.GPG()
encrypted_file_path = os.path.expanduser('~/Repositorios/binance_trader/.env.gpg')
with open(encrypted_file_path, 'rb') as file:
    datos = gpg.decrypt_file(file, passphrase=keyring.get_password("GPG_Passphrase", "gpg_python"))
if datos.ok:
    env_vars = dict(line.decode('utf-8').split('=', 1) for line in datos.data.splitlines())
    api_key = env_vars.get('API_KEY')
    secret_key = env_vars.get('SECRET_KEY')
# Inicializa el cliente de Binance
client = Spot(api_key, secret_key)

In [24]:
# Obtén información sobre el intercambio
exchange_info = client.exchange_info()
# Filtra los pares de trading contra USDT
usdt_pairs = [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['quoteAsset'] == 'USDT']

In [25]:
def get_historical_data(symbols, interval='1d'):
    data_frames = {}
    
    for symbol in symbols:
        start_time = 0
        data = []
        
        while True:
            # Obtener datos de klines
            klines = client.klines(symbol=symbol, interval=interval, limit=1000, startTime=start_time)
            # Si no hay más datos, salir del bucle
            if not klines:
                break
            for kline in klines:
                data.append({
                    'open_time': kline[0],
                    'open': float(kline[1]),
                    'high': float(kline[2]),
                    'low': float(kline[3]),
                    'close': float(kline[4]),
                    'volume': float(kline[5]),
                    'quote_asset_volume': float(kline[7]),
                    'number_of_trades': kline[8],
                    'taker_buy_base_asset_volume': float(kline[9]),
                    'taker_buy_quote_asset_volume': float(kline[10])
                })
            # Actualizar start_time para la próxima solicitud
            start_time = klines[-1][0] + 1
        
        data_frames[symbol] = pd.DataFrame(data)
    # Crear el dataframe multinivel
    df_list = []
    for symbol, df in data_frames.items():
        df = df.set_index('open_time')
        df.index = pd.to_datetime(df.index, unit='ms')
        df.columns = pd.MultiIndex.from_product([[symbol], df.columns])
        df_list.append(df)
    final_df = pd.concat(df_list, axis=1)
    return final_df


In [35]:
portfolio = ['XRPUSDT', 'DOTUSDT', 'LINKUSDT', 'HBARUSDT', 'XLMUSDT']
df = get_historical_data(portfolio)
df = df.dropna()